## Import

In [1]:
import pandas as pd
# from collections import Counter
import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.pyplot import figure
from time import strptime
import datetime
from datetime import datetime
# import plotly.express as px
# import matplotlib.mlab as mlab
# from statistics import mean
# import pylab
import math
import warnings
warnings.filterwarnings('ignore')
import time
import csv
import glob
import logging
import importlib
import tweets_utils
import json
%autosave 3600

Autosaving every 3600 seconds


## Indice
- [Profiles](#profiles)
- [CCDH and tweets process](#ccdh)

## Profiles
<a id="profiles"/>

In [2]:
# from platform import python_version
# print(python_version())

In [3]:
files = glob.glob("F:/COVID-19-Tweets/*/users.csv")
files

['F:/COVID-19-Tweets\\2020-01\\users.csv',
 'F:/COVID-19-Tweets\\2020-02\\users.csv',
 'F:/COVID-19-Tweets\\2020-03\\users.csv',
 'F:/COVID-19-Tweets\\2020-04\\users.csv',
 'F:/COVID-19-Tweets\\2020-05\\users.csv']

In [4]:
for chunk in pd.read_csv(files[0], chunksize=10, lineterminator = '\n'):
    break
# chunk.head()

In [5]:
chunksize = int(1e6)
flag = False
profile_n = 0
i = 0
profile_id = []
profile_name = []
# profile_loc = []
# profile_verified = []
# profile_creation = []
# default_profile_image = []
# statuses_count = []

users = []

# workers = 2
cols = [u'id', u'screen_name', u'location', u'created_at', u'verified', u'statuses_count', u'default_profile_image'] 
cont = 1

In [6]:
result = []
importlib.reload(tweets_utils)
start_time = time.perf_counter()
j = 0
for filename in files:
    print(f"Reading file: {filename}")
    result.extend(tweets_utils.process_all_data(filename, cols, flag, None))
    lista = result[j]
    j = j + 1
    print(f"Mese numero {j}")
    for i in range(len(lista)):
        profile_name.extend(lista[i]["users"])
        profile_id.extend(lista[i]["ids"])
stop_time = time.perf_counter()
print("Time: ",stop_time-start_time)

Reading file: F:/COVID-19-Tweets\2020-01\users.csv
Processing chunk 1
Processing chunk 2
Mese numero 1
Reading file: F:/COVID-19-Tweets\2020-02\users.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Mese numero 2
Reading file: F:/COVID-19-Tweets\2020-03\users.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Mese numero 3
Reading file: F:/COVID-19-Tweets\2020-04\users.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Mese numero 4
Reading file: F:/COVID-19-Tweets\2020-05\users.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Processing chunk 7
Processing chunk 8
Processing chunk 9
Mese numero 5
Time:  175.24603620000002


In [7]:
del result
del lista
del chunk
profile_name = pd.Series(profile_name)

## CCDH Research
<a id="ccdh"/>

In [5]:
disinform = pd.read_csv("C:/Users/Gianluca/Desktop/Supsi/Git/BachelorProject/csv/disinformation_users.csv")
disinform

,screen_name,status
0,mercola,active
1,RobertKennedyJr,active
2,TyCharleneB,removed
3,BusyDrT,suspended
4,IslamRizza,removed
5,DrButtar,active
6,unhealthytruth,active
7,sayerjigmi,active
8,KellyBroganMD,active
9,DrChrisNorthrup,active


In [13]:
names = []
status = []
for i in range(len(disinform["screen_name"])):
    print(disinform["screen_name"][i])
    if(disinform["screen_name"][i] in profile_name.values):
        names.append(disinform["screen_name"][i])
        status.append(disinform["status"][i])
print(names)
print(status)

mercola
RobertKennedyJr
TyCharleneB
BusyDrT
IslamRizza
DrButtar
unhealthytruth
sayerjigmi
KellyBroganMD
DrChrisNorthrup
DrBenTapper1
kevdjenkins1
['mercola', 'RobertKennedyJr', 'BusyDrT', 'DrButtar', 'unhealthytruth', 'DrChrisNorthrup', 'DrBenTapper1', 'kevdjenkins1']
['active', 'active', 'suspended', 'active', 'active', 'active', 'suspended', 'active']


In [14]:
with open("C:/Users/Gianluca/Desktop/Supsi/Git/BachelorProject/csv/users_ccdh.csv", 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["screen_name","status"])
    for i in range(len(names)):
        writer.writerow([names[i], status[i]])

In [6]:
users_link = glob.glob("C:/Users/Gianluca/Desktop/Supsi/Git/BachelorProject/csv/users_ccdh.csv")
users_disinform = pd.read_csv(users_link[0])
users_disinform

,screen_name,status
0,mercola,active
1,RobertKennedyJr,active
2,BusyDrT,suspended
3,DrButtar,active
4,unhealthytruth,active
5,DrChrisNorthrup,active
6,DrBenTapper1,suspended
7,kevdjenkins1,active


In [7]:
files_tweets = glob.glob("F:/COVID-19-Tweets/*/tweets.csv")
files_tweets

['F:/COVID-19-Tweets\\2020-01\\tweets.csv',
 'F:/COVID-19-Tweets\\2020-02\\tweets.csv',
 'F:/COVID-19-Tweets\\2020-03\\tweets.csv',
 'F:/COVID-19-Tweets\\2020-04\\tweets.csv',
 'F:/COVID-19-Tweets\\2020-05\\tweets.csv']

In [8]:
for chunk in pd.read_csv(files_tweets[0], chunksize=1000, lineterminator = '\n'):
    break
# chunk.info()

In [48]:
def found(name, list_name=users_disinform["screen_name"]):
    for i in list_name:
        if(name == i):
            return True
    return False

In [49]:
retweet_name = []
retweet_id = []
retweeted_name = []
retweeted_id = []
rt_link = []

tweet_name = []
tweet_id = []

reply_id = []
reply_name = []
replied_id = []
replied_name = []

link = []
result = []

total_len = 0
original_len = 0
retweet_len = 0
reply_len = 0


cols_tweets = [u'user_screen_name',u'user_id', u'created_at', 
               u'urls', u"rt_created_at", u"rt_user_id", 
               u"rt_user_screen_name", u'in_reply_to_user_id', 
               u'in_reply_to_screen_name', u'in_reply_to_status_id']

files_tweets = glob.glob("F:/COVID-19-Tweets/*/tweets.csv")

In [50]:
importlib.reload(tweets_utils)
start_time = time.perf_counter()
j = 0
for filename in files_tweets:
    print(f"Reading file: {filename}")
    result.extend(tweets_utils.process_all_data(filename, cols_tweets, True, users_disinform["screen_name"]))
    lista = result[j]
    j = j + 1
    print(f"Mese numero {j}")
#     x = 0
#     y = 0
    for i in range(len(lista)):
        #user who retweet         
        retweet_name.extend(lista[i]["retweet_users"])
        retweet_id.extend(lista[i]["retweet_ids"])
        #user retweeted
        retweeted_name.extend(lista[i]["retweeted_users"])
        retweeted_id.extend(lista[i]["retweeted_ids"])
        rt_link.extend(lista[i]["rt_link"])
        #user who reply
        reply_id.extend(lista[i]["reply_ids"])
        reply_name.extend(lista[i]["reply_users"])
        #use who get reply
        replied_id.extend(lista[i]["replied_ids"])
        replied_name.extend(lista[i]["replied_users"])
        #original tweet
        tweet_name.extend(lista[i]["users"])
        tweet_id.extend(lista[i]["ids"])
        link.extend(lista[i]["link"])
        #len
        total_len = total_len + lista[i]["total_len"]
        original_len = original_len + lista[i]["original_len"]
        retweet_len = retweet_len + lista[i]["retweet_len"]
        reply_len = reply_len + lista[i]["reply_len"]
#         data.append(lista[i]["data"])
#         print(i)
#         print("-")
#         if(i == 0):
#             x = 0
#         else:
#             x = y
#         y = x + len(lista[i]["users"])
#         for q in range(x,y):
#             if(found(lista[i]["users"][q])):          
    break
stop_time = time.perf_counter()
print("Time: ",stop_time-start_time)

Reading file: F:/COVID-19-Tweets\2020-01\tweets.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Mese numero 1
Time:  66.55623350000042


In [51]:
del result
del lista

In [52]:
def stats(total_len, original_len, retweet_len, reply_len):
    print(f'Number of total tweets: {total_len}')
    print(f'Number of original tweets: {original_len}')
    print(f'Number of retweet: {retweet_len}')
    print(f'Number of reply: {reply_len}')

    perc_original = np.around(original_len*100/total_len,2)
    perc_retweet = np.around(retweet_len*100/total_len,2)
    perc_reply = np.around(reply_len*100/total_len,2)
    print(f'Number of original_tweets: {perc_original}% of total tweets')
    print(f'Number of retweets: {perc_retweet}% of total tweets')
    print(f'Number of replies: {perc_reply}% of total tweets')

    print('Check sum == len(tweets): ',original_len + retweet_len + reply_len == total_len)

In [53]:
stats(total_len, original_len, retweet_len, reply_len)

Number of total tweets: 4082898
Number of original tweets: 704706
Number of retweet: 3175177
Number of reply: 203015
Number of original_tweets: 17.26% of total tweets
Number of retweets: 77.77% of total tweets
Number of replies: 4.97% of total tweets
Check sum == len(tweets):  True


In [54]:
disinform_rt_name = []
disinform_rt_id = []
rt_name = []
rt_id = []
link_rt = []

disinform_replied_name = []
disinform_replied_id = []
rp_name = []
rp_id = []

disinform_name = []
disinform_id = []
link_original = []

d_total_len = 0
d_original_len = 0
d_retweet_len = 0
d_reply_len = 0

In [55]:
print("process retweet")
#retweet
for i in range(len(retweeted_name)):
    if(found(retweeted_name[i])):
        d_total_len = d_total_len + 1
        d_retweet_len = d_retweet_len + 1
        
        disinform_rt_name.append(retweeted_name[i])
        disinform_rt_id.append(retweeted_id[i])
        
        rt_name.append(retweet_name[i])
        rt_id.append(retweet_id[i])
        
        link_tweet.append(rt_link[i])
    
del retweet_name
del retweet_id
del retweeted_name
del retweeted_id
del rt_link

print("process reply")
#reply
for i in range(len(replied_name)):
    if(found(replied_name[i])):
        d_total_len = d_total_len + 1
        d_reply_len = d_reply_len + 1
        
        disinform_replied_name.append(replied_name[i])
        disinform_replied_id.append(replied_id[i])
        
        rp_name.append(reply_name[i])
        rp_id.append(reply_id[i])
        
del reply_id
del reply_name
del replied_id
del replied_name

print("process original")
#original
for i in range(len(tweet_name)):
    if(found(tweet_name[i])):
        d_total_len = d_total_len + 1
        d_original_len = d_original_len + 1
        
        disinform_name.append(tweet_name[i])
        disinform_id.append(tweet_id[i])
        
        link.append(link[i])

del link
del tweet_name
del tweet_id

process retweet
process reply
process original


In [56]:
stats(d_total_len, d_original_len, d_retweet_len, d_reply_len)

Number of total tweets: 263
Number of original tweets: 13
Number of retweet: 240
Number of reply: 10
Number of original_tweets: 4.94% of total tweets
Number of retweets: 91.25% of total tweets
Number of replies: 3.8% of total tweets
Check sum == len(tweets):  True


In [102]:
urls = [tweets_utils.url_decompress(v) if v != "[]" else "0" for v in link]
date = [tweets_utils.process_datetime(v) for v in data]